In [1]:
#!pip3 freeze > "../requirements.txt"
#!pip3 install -r "../requirements.txt"  # giving some error
from google.cloud import bigquery
import pandas as pd
import numpy as np
import pandas_gbq
import copy
import copy
import pickle
import datetime as dt
from datetime import timezone
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans
from pins import board_rsconnect
from collections import Counter
from IPython.display import display
from datetime import timedelta
import pytz

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500) 

def GBQ_data(query_string):
    client = bigquery.Client('turing-230020')
    query = client.query(query_string)
    results = query.result()
    return results.to_dataframe()

In [2]:
## Will add this in r-pins 
pckt_sent = """SELECT job_id, count(distinct id) num_pckt_sent  FROM `turing-230020.devdb_mirror.ms2_job_packet` 
WHERE packet_status_id=2 group by 1 order by 2 desc"""
pckt_sent = """SELECT job_id, count(distinct packet_id) num_pckt_sent FROM `turing-230020.analytics_views.job_dev_pairs`
where packet_id is not null group by 1"""
pckt_sent = GBQ_data(pckt_sent)
pckt_sent.shape

(5966, 2)

In [3]:
cluster_data = """Select * from turing-dev-337819.pdsa.demand_cluster"""
cluster_data =  GBQ_data(cluster_data)
cluster_data_ets = cluster_data.loc[cluster_data['client_category']=='1.Platinum']
cluster_data_fss = cluster_data.loc[cluster_data['client_category']!='1.Platinum']

API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
job_data =board.pin_read("muhammad_usman/demand-basic-data")


In [4]:
#final_data.loc[final_data['active']==1 ,['job_id', 'number_of_open_roles', 'total_positions', 'total_open_positions__c']].isnull().sum()
#final_data.loc[final_data['active']==1 ,'total_positions'].value_counts()

In [5]:
job_data_cols = ['job_id', 'company', 'client_category','opportunity_created_date', 'dc_date', 'max_acceptable_rate', 'is_deleted',
                 'job_value', 'status', 'num_must_have_skills', 'must_have_total_years_of_experience',
                'ms_num_queries', 'ss_queries', 'queries', 'interviews_requested', 'interviews_scheduled',
                'interviews_happened','interviews_passed', 'interviews_failed', 'region', 'engagement_days',
                'number_of_open_roles', 'total_positions', 'total_open_positions__c', 'must_have_skill_ids', 'must_have_skill_names']

final_data = job_data[job_data_cols].copy()
final_data['opportunity_created_date'] = pd.to_datetime(final_data['opportunity_created_date'])
final_data['dc_date'] = pd.to_datetime(final_data['dc_date'])
final_data['dc'] = np.where(final_data['dc_date'].isna(), 0,1)
final_data['ninety_days_old'] = np.where((final_data['opportunity_created_date'] > dt.datetime.now(pytz.utc) - timedelta(days = 90)) & (final_data['dc']==0) , 1,0)
final_data['time_taken_DC'] = (final_data['dc_date'] - final_data['opportunity_created_date']).dt.days
#final_data['time_taken_DC'] = np.where(final_data['time_taken_DC']<0, 0, final_data['time_taken_DC'])
final_data['enterprise'] = np.where(final_data['client_category']=='1.Platinum', 1,0)
final_data['active'] = np.where(~(final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused'])) & (final_data['dc']==0), 1,0)
final_data['active_roles'] = np.where(final_data['active']==1, final_data['total_positions'],0)
final_data['lost'] = np.where((final_data['status'].isin(['Lost Opportunity', 'Project Aborted', 'Paused']) & (final_data['dc']==0)), 1,0)
final_data['max_acceptable_rate'] = np.where(final_data['max_acceptable_rate'] >300 , 300, final_data['max_acceptable_rate'])
final_data['total_positions'] = np.where(final_data['total_positions']==0 , 1,final_data['total_positions'])
final_data['is_bulk'] = np.where(final_data['total_positions']>2 , 1,0)
final_data = final_data.merge(pckt_sent, how='left', on='job_id')
final_data['is_pckt_sent'] = np.where(final_data['num_pckt_sent'].isna(), 0,1)

In [6]:
overall_data = final_data.merge(cluster_data[['job_id', 'cluster']], how='right', on='job_id')
ets = final_data.loc[final_data['enterprise']==1].merge(cluster_data_ets[['job_id', 'cluster']], how='right', on='job_id')
fss = final_data.loc[final_data['enterprise']==0].merge(cluster_data_fss[['job_id', 'cluster']], how='right', on='job_id')


In [7]:
#np.mean(ets.time_taken_DC.value_counts())
#tr = final_data.loc[final_data['dc']==1,'dc_date'] - final_data.loc[final_data['dc']==1,'opportunity_created_date']
#tr.dt.days.value_counts()

#fss.groupby(['cluster','region'], dropna=False).agg({'job_id':'count','dc':'sum', 'active':'sum'}).reset_index()
#ets.loc[(ets['cluster']==0) & ets['active']==1]

In [8]:
#display(fss.groupby('cluster').agg({'total_positions':'sum'}).reset_index())
#display(fss.loc[fss['is_bulk']==1].groupby('cluster').agg({'total_positions':'sum'}).reset_index().rename(columns={'total_positions':'total_positions_bulk'}))

In [9]:
aaa = ets.groupby('cluster').agg({'job_id':'count', 'is_bulk' : 'sum', 'dc':'sum',  'num_must_have_skills':'mean',
                            'max_acceptable_rate':'sum', 'active':'sum', 'lost':'sum',
                            'active_roles':'sum', 'ms_num_queries':'sum',
                            'ss_queries' : 'sum', 'queries' : 'sum', 'interviews_requested':'sum',
                            'interviews_scheduled':'sum', 'interviews_happened':'sum',
                            'interviews_passed':'sum', 'interviews_failed':'sum',
                           'ninety_days_old':'sum', 'job_value':'sum',
                                 'engagement_days':'sum', 'num_pckt_sent':'sum'}).reset_index()
aaa

,cluster,job_id,is_bulk,dc,num_must_have_skills,max_acceptable_rate,active,lost,active_roles,ms_num_queries,ss_queries,queries,interviews_requested,interviews_scheduled,interviews_happened,interviews_passed,interviews_failed,ninety_days_old,job_value,engagement_days,num_pckt_sent
0,0,191,10,135,1.010471,5275.0,8,48,80.0,641.0,245.0,886.0,294.0,218.0,208.0,9.0,53.0,15,4647.3,11572.0,163
1,1,788,54,401,2.093909,24251.0,56,331,103.0,6607.0,1648.0,8255.0,1515.0,1054.0,1023.0,70.0,513.0,94,18741.3,64361.0,1338
2,2,511,54,150,3.375734,19120.0,50,311,123.0,7398.0,1821.0,9219.0,1219.0,792.0,748.0,47.0,505.0,64,15950.8,19676.0,1199


In [10]:
## Supply Quality issue
# IS average IS wehre DC is 1

ir_sec = ets.loc[ets['dc']==1].groupby('cluster').agg({'job_id':'count','interviews_scheduled':'sum', 'interviews_requested':'sum'}).reset_index()
ir_sec['expercted_interview_sec'] = round(ir_sec['interviews_scheduled']/ir_sec['job_id'],0)
ir_sec['expercted_ir'] = round(ir_sec['interviews_requested']/ir_sec['job_id'],0)
ir_sec = ir_sec[['cluster', 'expercted_interview_sec', 'expercted_ir']]
ir_sec

,cluster,expercted_interview_sec,expercted_ir
0,0,1.0,1.0
1,1,2.0,2.0
2,2,3.0,4.0


In [11]:
supply_issue = ets.loc[ets['lost']==1].merge(ir_sec, how='left', on='cluster')
supply_issue['is_qulaity_issue'] = np.where(supply_issue['interviews_scheduled'] >= supply_issue['expercted_interview_sec'], 1,0)
supply_issue['is_volume_issue'] = np.where(supply_issue['interviews_requested'] >= supply_issue['expercted_ir'], 1,0)

In [12]:
display(supply_issue.groupby(['cluster','is_qulaity_issue']).agg({'job_id':'count'}).reset_index())
supply_issue.groupby(['cluster','is_volume_issue']).agg({'job_id':'count'}).reset_index()



,cluster,is_qulaity_issue,job_id
0,0,0,41
1,0,1,7
2,1,0,272
3,1,1,59
4,2,0,276
5,2,1,35


,cluster,is_volume_issue,job_id
0,0,0,34
1,0,1,14
2,1,0,236
3,1,1,95
4,2,0,270
5,2,1,41


In [13]:
#ets.loc[ets['dc']==1].groupby('cluster').agg({'job_id':'count','interviews_scheduled':'mean'}).reset_index()

In [14]:
aaa = ets.groupby('cluster').agg({'job_id':'count', 'is_bulk' : 'sum', 'dc':'sum',  'num_must_have_skills':'mean',
                            'max_acceptable_rate':'sum', 'active':'sum', 'lost':'sum',
                            'active_roles':'sum', 'ms_num_queries':'sum',
                            'ss_queries' : 'sum', 'queries' : 'sum', 'interviews_requested':'sum',
                            'interviews_scheduled':'sum', 'interviews_happened':'sum',
                            'interviews_passed':'sum', 'interviews_failed':'sum',
                           'ninety_days_old':'sum', 'job_value':'sum',
                                 'engagement_days':'sum', 'num_pckt_sent':'sum'}).reset_index()


bbb = pd.DataFrame()
bbb['cluster'] = aaa['cluster']
bbb['Number of Jobs'] = aaa['job_id']
bbb['Bulk %'] = round(100*aaa['is_bulk']/aaa['job_id'],1)
bbb['Number of DC'] = aaa['dc']
bbb['DC%'] = round(100*aaa['dc']/aaa['job_id'],1)
bbb['Active Jobs'] = aaa['active']
bbb['Active Positions'] = aaa['active_roles']
bbb['Active Jobs %'] = round(100*aaa['active']/aaa['job_id'],1)
bbb['Lost Jobs %'] = round(100*aaa['lost']/aaa['job_id'],1)
bbb['90 Days Old Jobs with no DC %'] = round(100*aaa['ninety_days_old']/aaa['job_id'],1)
bbb['Number of must have skills'] = round(aaa['num_must_have_skills'],1)
bbb['Pay Rate'] = round(aaa['max_acceptable_rate']/aaa['job_id'],1)
bbb['Interviews requested'] =  round(aaa['interviews_requested']/aaa['job_id'],1)
bbb['Interviews scheduled'] =  round(aaa['interviews_scheduled']/aaa['job_id'],1)
bbb['Interviews happened'] =  round(aaa['interviews_happened']/aaa['job_id'],1)
bbb['Interviews passed'] =  round(aaa['interviews_passed']/aaa['job_id'],2)
bbb['Interviews failed'] =  round(aaa['interviews_failed']/aaa['job_id'],2)
bbb['MS Queries'] =  round(aaa['ms_num_queries']/aaa['job_id'],1)
bbb['SS Queries'] =  round(aaa['ss_queries']/aaa['job_id'],1)
bbb['Queries'] =  round(aaa['queries']/aaa['job_id'],1)
bbb['Job Vlaue'] =  round(aaa['job_value']/aaa['job_id'],1)
bbb['Engagement Days'] =  round(aaa['engagement_days']/aaa['job_id'],1)
bbb['Num Packet Sent'] =  round(aaa['num_pckt_sent']/aaa['job_id'],1)

#bbb.to_csv('ets_summary.csv')
bbb


,cluster,Number of Jobs,Bulk %,Number of DC,DC%,Active Jobs,Active Positions,Active Jobs %,Lost Jobs %,90 Days Old Jobs with no DC %,Number of must have skills,Pay Rate,Interviews requested,Interviews scheduled,Interviews happened,Interviews passed,Interviews failed,MS Queries,SS Queries,Queries,Job Vlaue,Engagement Days,Num Packet Sent
0,0,191,5.2,135,70.7,8,80.0,4.2,25.1,7.9,1.0,27.6,1.5,1.1,1.1,0.05,0.28,3.4,1.3,4.6,24.3,60.6,0.9
1,1,788,6.9,401,50.9,56,103.0,7.1,42.0,11.9,2.1,30.8,1.9,1.3,1.3,0.09,0.65,8.4,2.1,10.5,23.8,81.7,1.7
2,2,511,10.6,150,29.4,50,123.0,9.8,60.9,12.5,3.4,37.4,2.4,1.5,1.5,0.09,0.99,14.5,3.6,18.0,31.2,38.5,2.3


In [15]:
aaa = fss.groupby('cluster').agg({'job_id':'count', 'is_bulk' : 'sum', 'dc':'sum',  'num_must_have_skills':'mean',
                            'max_acceptable_rate':'sum', 'active':'sum', 'lost':'sum',
                            'active_roles':'sum', 'ms_num_queries':'sum',
                            'ss_queries' : 'sum', 'queries' : 'sum', 'interviews_requested':'sum',
                            'interviews_scheduled':'sum', 'interviews_happened':'sum',
                            'interviews_passed':'sum', 'interviews_failed':'sum',
                           'ninety_days_old':'sum', 'job_value':'sum',
                                 'engagement_days':'sum', 'num_pckt_sent':'sum'}).reset_index()

bbb = pd.DataFrame()
bbb['cluster'] = aaa['cluster']
bbb['Number of Jobs'] = aaa['job_id']
bbb['Bulk %'] = round(100*aaa['is_bulk']/aaa['job_id'],1)
bbb['Number of DC'] = aaa['dc']
bbb['DC%'] = round(100*aaa['dc']/aaa['job_id'],1)
bbb['Active Jobs'] = aaa['active']
bbb['Active Positions'] = aaa['active_roles']
bbb['Active Jobs %'] = round(100*aaa['active']/aaa['job_id'],1)
bbb['Lost Jobs %'] = round(100*aaa['lost']/aaa['job_id'],1)
bbb['90 Days Old Jobs with no DC %'] = round(100*aaa['ninety_days_old']/aaa['job_id'],1)
bbb['Number of must have skills'] = round(aaa['num_must_have_skills'],1)
bbb['Pay Rate'] = round(aaa['max_acceptable_rate']/aaa['job_id'],1)
bbb['Interviews requested'] =  round(aaa['interviews_requested']/aaa['job_id'],1)
bbb['Interviews scheduled'] =  round(aaa['interviews_scheduled']/aaa['job_id'],1)
bbb['Interviews happened'] =  round(aaa['interviews_happened']/aaa['job_id'],1)
bbb['Interviews passed'] =  round(aaa['interviews_passed']/aaa['job_id'],2)
bbb['Interviews failed'] =  round(aaa['interviews_failed']/aaa['job_id'],2)
bbb['MS Queries'] =  round(aaa['ms_num_queries']/aaa['job_id'],1)
bbb['SS Queries'] =  round(aaa['ss_queries']/aaa['job_id'],1)
bbb['Queries'] =  round(aaa['queries']/aaa['job_id'],1)
bbb['Job Vlaue'] =  round(aaa['job_value']/aaa['job_id'],1)
bbb['Engagement Days'] =  round(aaa['engagement_days']/aaa['job_id'],1)
bbb['Num Packet Sent'] =  round(aaa['num_pckt_sent']/aaa['job_id'],1)

#bbb.to_csv('fss_summary.csv')
bbb


,cluster,Number of Jobs,Bulk %,Number of DC,DC%,Active Jobs,Active Positions,Active Jobs %,Lost Jobs %,90 Days Old Jobs with no DC %,Number of must have skills,Pay Rate,Interviews requested,Interviews scheduled,Interviews happened,Interviews passed,Interviews failed,MS Queries,SS Queries,Queries,Job Vlaue,Engagement Days,Num Packet Sent
0,0,732,0.8,400,54.6,56,56.0,7.7,37.7,10.7,2.5,30.9,2.2,1.4,1.3,0.10,0.49,4.1,2.7,6.7,8.6,72.2,1.3
1,1,1776,1.5,456,25.7,104,105.0,5.9,68.5,9.8,3.1,34.0,2.1,1.0,0.9,0.07,0.43,5.0,7.2,12.2,14.6,34.0,1.3
2,2,3277,1.2,672,20.5,165,178.0,5.0,74.5,10.2,2.8,33.0,1.7,0.8,0.7,0.05,0.33,2.9,7.8,10.7,7.6,26.3,1.1


# Positons based DC / Child parent relation

## Cluster attributes for a skill set

In [15]:
overall_data = overall_data.loc[~overall_data['must_have_skill_names'].isna()]
overall_data['must_have_skill_names_list'] = overall_data['must_have_skill_names'].apply(lambda x:x.split(',', -1))

overall_data['must_have_skill_names_list_sort'] = ''
for i in overall_data.index:
    overall_data.at[i , 'must_have_skill_names_list_sort'] = sorted(overall_data.at[i , 'must_have_skill_names_list'])

def jobs_subset_skill(skill):
    overall_data[skill] = 0
    for ind in overall_data.index:
        if skill in overall_data.at[ind, 'must_have_skill_names_list_sort']:
            overall_data.at[ind,skill] = 1
    

In [16]:
aaa = fss.groupby('cluster').agg({'job_id':'count', 'dc':'sum',  'num_must_have_skills':'mean',
                            'max_acceptable_rate':'sum', 'active':'sum', 'active_roles':'sum', 'lost':'sum', 'ms_num_queries':'sum',
                            'ss_queries' : 'sum', 'queries' : 'sum', 'interviews_requested':'sum',
                            'interviews_scheduled':'sum', 'interviews_happened':'sum',
                            'interviews_passed':'sum', 'interviews_failed':'sum',
                           'ninety_days_old':'sum', 'job_value':'sum'}).reset_index()
bbb = pd.DataFrame()
bbb['cluster'] = aaa['cluster']
bbb['Number of Jobs'] = aaa['job_id']
bbb['Number of DC'] = aaa['dc']
bbb['DC'] = round(100*aaa['dc']/aaa['job_id'],1)
bbb['Active Jobs'] = aaa['active']
bbb['Active Positions'] = aaa['active_roles']
bbb['Active Jobs %'] = round(100*aaa['active']/aaa['job_id'],1)
bbb['Lost Jobs %'] = round(100*aaa['lost']/aaa['job_id'],1)
bbb['90 Days Old Jobs with no DC %'] = round(100*aaa['ninety_days_old']/aaa['job_id'],1)
bbb['Number of must have skills'] = round(aaa['num_must_have_skills'],1)
bbb['Pay Rate'] = round(aaa['max_acceptable_rate']/aaa['job_id'],1)
bbb['Interviews requested'] =  round(aaa['interviews_requested']/aaa['job_id'],1)
bbb['Interviews scheduled'] =  round(aaa['interviews_scheduled']/aaa['job_id'],1)
bbb['Interviews happened'] =  round(aaa['interviews_happened']/aaa['job_id'],1)
bbb['Interviews passed'] =  round(aaa['interviews_passed']/aaa['job_id'],2)
bbb['Interviews failed'] =  round(aaa['interviews_failed']/aaa['job_id'],2)
bbb['MS Queries'] =  round(aaa['ms_num_queries']/aaa['job_id'],1)
bbb['SS Queries'] =  round(aaa['ss_queries']/aaa['job_id'],1)
bbb['Queries'] =  round(aaa['queries']/aaa['job_id'],1)
bbb['Job Vlaue'] =  round(aaa['job_value']/aaa['job_id'],1)

#bbb.to_csv('fss_summary.csv')
bbb

,cluster,Number of Jobs,Number of DC,DC,Active Jobs,Active Positions,Active Jobs %,Lost Jobs %,90 Days Old Jobs with no DC %,Number of must have skills,Pay Rate,Interviews requested,Interviews scheduled,Interviews happened,Interviews passed,Interviews failed,MS Queries,SS Queries,Queries,Job Vlaue
0,0,732,400,54.6,56,56.0,7.7,37.7,10.7,2.5,30.9,2.2,1.4,1.3,0.10,0.49,4.1,2.7,6.7,8.6
1,1,1776,456,25.7,104,105.0,5.9,68.5,9.8,3.1,34.0,2.1,1.0,0.9,0.07,0.43,5.0,7.2,12.2,14.6
2,2,3277,672,20.5,165,178.0,5.0,74.5,10.2,2.8,33.0,1.7,0.8,0.7,0.05,0.33,2.9,7.8,10.7,7.6


In [131]:
#API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
#SERVER = 'https://rstudio-connect.turing.com/'
#board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
#data = board.pin_read("muhammad_usman/demand-avs-data")
#data.loc[data['must_have_skill_names']=="['Python']"]
#data.loc[data['must_have_skill_names']=="['Go/Golang']"]

## not using 


job_mcq_must = """
SELECT 
    job.job_id, 
    job.skill_id, 
    skill.skill_name
from `turing-230020.devdb_mirror.ms2_job_skill` job
LEFT join `turing-230020.devdb_mirror.base_all_skills_v4` skill on job.skill_id=skill.id
where job.job_skill_level_id=1
"""

job_mcq_must = GBQ_data(job_mcq_must)


ff = overall_data.loc[(overall_data['active']==1) & (overall_data['company']!='OpenAI')]
ff = ff[['job_id', 'total_positions', 'total_open_positions__c', 'must_have_skill_names', 'dc']].merge(job_mcq_must, how='left', on='job_id')
ff['total_open_positions__c'] = ff['total_open_positions__c'].fillna(1)
ff = ff.groupby(['skill_id', 'skill_name']).agg({'total_open_positions__c':'sum'}).reset_index()


API_KEY = 'yDAssnMUtqatxoOpyNgYVKZcXfLP3vwD' 
SERVER = 'https://rstudio-connect.turing.com/'
board = board_rsconnect(server_url=SERVER, api_key=API_KEY)
dev_data = board.pin_read("muhammad_usman/devs-skill-pass-list")
dev_data['last_update_availability'] = pd.to_datetime(dev_data['last_update_availability'])
dev_data = dev_data.loc[(dev_data['num_passed_skill']>0) & (~dev_data['num_skill'].isna())]
dev_data = dev_data.loc[(dev_data['vetting_status'].isin(['available supply','ready 2.0','passed ti','standardized resume']))].reset_index(drop=True).copy()
dev_data['self_skill_id_list'] = dev_data.self_skill_id.apply(lambda x:list(map(int, x.split(',',-1))))
dev_data['passed_skill_id_list'] = dev_data.passed_skill_id.apply(lambda x:list(map(int, x.replace('[', '').replace(']','').split(',',-1))))
dev_data['passed_skill_id_set'] = dev_data.passed_skill_id_list.apply(lambda x:set(x))
dev_data['self_skill_id_set'] = dev_data.self_skill_id_list.apply(lambda x:set(x))
dev_data.shape

def avs_supply_passed_mcq(skill):
    temp_data = dev_data                            
    for s in skill:
        temp_data = temp_data.loc[temp_data['passed_skill_id_set'].apply(lambda x:s in x)]
    return temp_data['cluster_label'].value_counts().append(pd.Series(temp_data.shape[0], index = ['total_devs']))

col_name = ['total_devs', 'Elite', 'High Quality', 'Average', 'Low Quality', 'Low Experience']
df_passed = pd.DataFrame(columns = col_name)
for i in ff.skill_id:
    df_passed = df_passed.append(avs_supply_passed_mcq([i]),ignore_index=True)

df_passed['skill_id'] = ff.skill_id
duck1 = ff.merge(df_passed, how='left', on='skill_id')#.to_csv('without_openai_active.csv')


pos = """with blah as (
SELECT
op.opportunityid,
op1.opportunityid as opportunityid2,
op.job_id,
op1.job_id as job_id2,
op.total_positions,
op1.total_positions as total_positions2
from `matchingmetrics.opps_positions` op
JOIN `matchingmetrics.opps_positions` op1 on op.opportunityid = op1.parent_opp_id
-- LEFT JOIN `curated.opportunity_value` ov on op.parent_opp_id = ov.opportunityid
),

data as(
select blah.*, ov.resource_chosen, ov.job_id as job_id_join from blah
LEFT JOIN `curated.opportunity_value` ov on blah.job_id2 = ov.job_id
),
job_id_data as(
SELECT job_id, count(*) total_pos, sum(resource_chosen) as dc
from data GROUP by 1 order by 2 desc)

SELECT * from job_id_data
left join turing-dev-337819.pdsa.demand_cluster using(job_id)
"""

pos_data = GBQ_data(pos)
pos_data.shape

pos_data_filter = pos_data.loc[~pos_data['cluster'].isna()].copy()
pos_data_filter['enterprise'] = np.where(pos_data_filter['client_category']=='1.Platinum', 1,0)

ff = overall_data.loc[(overall_data['active']==0) & (overall_data['company']!='OpenAI')]
ff = ff[['job_id', 'total_positions', 'total_open_positions__c', 'must_have_skill_names', 'dc']].merge(job_mcq_must, how='left', on='job_id')
ff['total_open_positions__c'] = ff['total_open_positions__c'].fillna(1)
duck2 = ff.groupby(['skill_id', 'skill_name']).agg({'total_open_positions__c':'sum', 'dc':'sum'}).reset_index()#.to_csv('without_openai_dc.csv')
#dc = ff.merge(pos_data_filter, how='left', on='job_id')
duck2.merge(duck1, on='skill_id', how='right').to_csv('all_single_skill.csv')